In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError

In [2]:
#Dataset
data = pd.read_csv("/content/Dataset-hasil-panen-padi-2018-2021.csv")
data.head()

,Periode,Provinsi,Luas Panen (Ha),Produksi Padi (Ton-GKG),Produktivitas (Ton/Ha)
0,2018-01-01,ACEH,14541.53,76118.47,5.23
1,2018-02-01,ACEH,27075.77,142876.76,5.28
2,2018-03-01,ACEH,66403.96,375841.91,5.66
3,2018-04-01,ACEH,58507.03,336742.80,5.76
4,2018-05-01,ACEH,16867.65,96372.36,5.71


In [3]:
#Memilih kolom yang diperlukan
data = data[['Periode', 'Luas Panen (Ha)', 'Produksi Padi (Ton-GKG)']]

In [4]:
#Ubah format Periode ke datetime
data['Periode'] = pd.to_datetime(data['Periode'])
type(data['Periode'])

pandas.core.series.Series

In [5]:
#Sortir data berdasarkan Periode
data = data.sort_values('Periode')
type(data)

pandas.core.frame.DataFrame

In [6]:
data['Luas Panen (Ha)'] = (data['Luas Panen (Ha)'] - data['Luas Panen (Ha)'].min()) / (data['Luas Panen (Ha)'].max() - data['Luas Panen (Ha)'].min())
data['Produksi Padi (Ton-GKG)'] = (data['Produksi Padi (Ton-GKG)'] - data['Produksi Padi (Ton-GKG)'].min()) / (data['Produksi Padi (Ton-GKG)'].max() - data['Produksi Padi (Ton-GKG)'].min())

type(data['Luas Panen (Ha)'])
type(data['Produksi Padi (Ton-GKG)'])


pandas.core.series.Series

In [7]:
#Pemisahan data latih dan data uji
train_data = data[data['Periode'].dt.year < 2021]
test_data = data[data['Periode'].dt.year == 2021]

In [8]:
#Persiapan data untuk model LSTM
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps, 1])  #Ambil kolom Luas Panen (Ha)
        y.append(data[i+n_steps, 2])  #Ambil kolom Produksi Padi (Ton-GKG)
    return np.array(X), np.array(y)

#Data latih dan uji disesuaikan
n_steps = 12  #Jumlah periode yang digunakan sebagai input (bulan)
X_train, y_train = prepare_data(train_data.values, n_steps)
X_test, y_test = prepare_data(test_data.values, n_steps)

In [9]:
from tensorflow.keras.layers import Dropout
#Membangun model LSTM
model = Sequential()
model.add(LSTM(256, activation='relu', input_shape=(n_steps, 1), return_sequences=True))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(182, activation='relu', return_sequences=True))
model.add(Dense(182, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [10]:
#Ubah tipe data ke float32
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)

In [11]:
#Latih model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=2)

Epoch 1/100
38/38 - 6s - loss: 0.0133 - 6s/epoch - 150ms/step
Epoch 2/100
38/38 - 2s - loss: 0.0130 - 2s/epoch - 64ms/step
Epoch 3/100
38/38 - 3s - loss: 0.0129 - 3s/epoch - 73ms/step
Epoch 4/100
38/38 - 3s - loss: 0.0129 - 3s/epoch - 71ms/step
Epoch 5/100
38/38 - 2s - loss: 0.0129 - 2s/epoch - 65ms/step
Epoch 6/100
38/38 - 2s - loss: 0.0129 - 2s/epoch - 61ms/step
Epoch 7/100
38/38 - 2s - loss: 0.0130 - 2s/epoch - 61ms/step
Epoch 8/100
38/38 - 3s - loss: 0.0129 - 3s/epoch - 79ms/step
Epoch 9/100
38/38 - 2s - loss: 0.0129 - 2s/epoch - 63ms/step
Epoch 10/100
38/38 - 2s - loss: 0.0129 - 2s/epoch - 61ms/step
Epoch 11/100
38/38 - 2s - loss: 0.0129 - 2s/epoch - 61ms/step
Epoch 12/100
38/38 - 2s - loss: 0.0129 - 2s/epoch - 62ms/step
Epoch 13/100
38/38 - 3s - loss: 0.0129 - 3s/epoch - 81ms/step
Epoch 14/100
38/38 - 2s - loss: 0.0129 - 2s/epoch - 62ms/step
Epoch 15/100
38/38 - 2s - loss: 0.0129 - 2s/epoch - 62ms/step
Epoch 16/100
38/38 - 2s - loss: 0.0128 - 2s/epoch - 62ms/step
Epoch 17/100
38/

In [12]:
#ubah tipe data ke float32
X_test = X_test.astype(np.float32)

# Prediksi produksi padi pada data uji
y_pred = model.predict(X_test)

13/13 [==============================] - 1s 19ms/step


In [13]:
#Inversi normalisasi data hasil prediksi dan data uji
y_pred = (y_pred * (data['Produksi Padi (Ton-GKG)'].max() - data['Produksi Padi (Ton-GKG)'].min())) + data['Produksi Padi (Ton-GKG)'].min()
y_test = (y_test * (data['Produksi Padi (Ton-GKG)'].max() - data['Produksi Padi (Ton-GKG)'].min())) + data['Produksi Padi (Ton-GKG)'].min()

In [14]:
from sklearn.metrics import mean_squared_error

#Hitung metrik evaluasi
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("MSE: ", mse)
print("RMSE: ", rmse)

MSE:  0.01319652124747549
RMSE:  0.1148761126060396


In [15]:
periode=('2022-04-03')
type(periode)

str

In [18]:
# Fungsi menyiapkan input data
def prepare_input_data(period, luas_panen):
    scaled_period = (period - data['Periode'].min()) / (data['Periode'].max() - data['Periode'].min())
    scaled_luas_panen = (luas_panen - data['Luas Panen (Ha)'].min()) / (data['Luas Panen (Ha)'].max() - data['Luas Panen (Ha)'].min())
    input_data = np.array([[scaled_period, scaled_luas_panen]])
    return input_data

# Contoh input manual untuk periode dan luas panen

input_period = '2022-04-03'
input_period = pd.to_datetime(input_period, format='%Y-%m-%d')  # Konversi ke tipe data timestamp
input_luas_panen = 40000.0

# Persiapan input data
input_data = prepare_input_data(input_period, input_luas_panen)

# Melakukan prediksi menggunakan model
y_pred = model.predict(input_data)

# Inversi normalisasi data hasil prediksi
predicted_output = (y_pred * (data['Produksi Padi (Ton-GKG)'].max() - data['Produksi Padi (Ton-GKG)'].min())) + data['Produksi Padi (Ton-GKG)'].min()

predicted_output_float = float(predicted_output[0])
predicted_output_round = round(predicted_output_float,2)

print("Predicted Produksi Padi (Ton-GKG):", predicted_output_round)


1/1 [==============================] - 0s 66ms/step
Predicted Produksi Padi (Ton-GKG): 124766.38


In [19]:
type(predicted_output_round)

float

In [20]:
type(input_luas_panen)

float

In [21]:
type(input_period)

pandas._libs.tslibs.timestamps.Timestamp

In [22]:
from tensorflow.keras.models import save_model

# Misalkan model Anda bernama "model"
model.save('/content/model_produksipadi.h5')

In [23]:
import tensorflow as tf
# Load the Keras model
model = tf.keras.models.load_model('/content/model_produksipadi.h5')

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('/content/model_produksipadi.tflite', 'wb') as f:
    f.write(tflite_model)

from google.colab import files
files.download('model_produksipadi.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>